In [ ]:
!pip install numpy opencv-python pillow tqdm requests

In [ ]:
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance
import textwrap
import os
from tqdm import tqdm
import requests
import zipfile
import io
import random
from functools import lru_cache

class TextAnimationVideoGenerator:
    def __init__(self,
                 width=1080,  # Standard shorts/vertical format width
                 height=1920, # Standard shorts/vertical format height
                 fps=30,
                 font_path="custom_font.ttf",
                 font_size=100,
                 text_color=(255, 255, 255),
                 bg_color=(0, 0, 0),
                 chars_per_frame=2,
                 scroll_speed=1,
                 margin_top=100,
                 margin_bottom=100,
                 margin_sides=50,
                 text_effects=True,
                 category_path=None):

        self.width = width
        self.height = height
        self.fps = fps
        self.text_color = text_color
        self.bg_color = bg_color
        self.chars_per_frame = chars_per_frame
        self.scroll_speed = scroll_speed
        self.margin_top = margin_top
        self.margin_bottom = margin_bottom
        self.margin_sides = margin_sides
        self.text_effects = text_effects
        self.category_path = category_path

        # Dynamic elements
        self.frame_count = 0

        # Visual style settings
        self.glow_amount = 3
        self.shadow_offset = 3
        self.shadow_color = (0, 0, 0, 180)  # Semi-transparent black

        # Try to load style from category if provided
        if category_path and os.path.exists(os.path.join(category_path, "style.txt")):
            font_path = self.load_style_from_category(category_path)

        # Load font
        self.font_size = font_size
        self.load_font(font_path, font_size)

        # Measure available drawing space
        self.usable_width = self.width - (2 * self.margin_sides)
        self.usable_height = self.height - self.margin_top - self.margin_bottom

        # Calculate line height based on actual font metrics
        test_text = "AygjpqQ|" # Text with ascenders and descenders
        bbox = self.font.getbbox(test_text)
        self.text_height = bbox[3] - bbox[1]
        self.line_height = int(self.text_height * 1.3)  # Add some spacing between lines

        # Calculate how many characters fit on one line
        test_width = self.font.getlength("m" * 10) / 10  # Average width of character 'm'
        self.chars_per_line = max(10, int(self.usable_width / test_width))

        # Calculate how many lines fit on screen
        self.lines_per_screen = self.usable_height // self.line_height

        print(f"Font metrics: text height={self.text_height}px, line height={self.line_height}px")
        print(f"Screen capacity: {self.chars_per_line} chars per line, {self.lines_per_screen} lines per screen")

        # Additional fonts for multi-font variation
        self.title_font = None
        try:
            title_size = int(font_size * 1.5)
            self.title_font = ImageFont.truetype(font_path, title_size)
        except Exception:
            self.title_font = self.font

        # Precompute backgrounds to improve performance
        self.background_cache = {}

    def load_style_from_category(self, category_path):
        """Load style information from category folder"""
        style_path = os.path.join(category_path, "style.txt")
        try:
            with open(style_path, 'r', encoding='utf-8') as file:
                font_url = file.read().strip()
                print(f"Loaded font URL from style file: {font_url}")
                return font_url
        except Exception as e:
            print(f"Error loading style file: {e}")
            return "custom_font.ttf"  # Default fallback

    def load_font(self, font_path, font_size):
        """Load font or download it if specified by URL"""
        try:
            # Check if it's a URL
            if font_path.startswith(('http://', 'https://')):
                font_path = self.download_font(font_path)

            self.font = ImageFont.truetype(font_path, font_size)
            print(f"Successfully loaded font: {font_path} at size {font_size}px")
        except Exception as e:
            print(f"Font loading error: {e}. Using default font.")
            # Use a system font that's likely to be available
            try:
                system_fonts = ['Arial.ttf', 'Verdana.ttf', 'DejaVuSans.ttf', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf']
                for font in system_fonts:
                    try:
                        self.font = ImageFont.truetype(font, font_size)
                        print(f"Using system font: {font}")
                        return
                    except:
                        continue
            except:
                pass

            # If all else fails, use default
            self.font = ImageFont.load_default()
            # Try to resize default font
            if hasattr(self.font, "size"):
                self.font = self.font.font_variant(size=font_size)

    def download_font(self, url):
        """Download a font from URL"""
        try:
            print(f"Downloading font from {url}...")

            # Handle Google Fonts URL format
            if "fonts.google.com/download" in url:
                # Extract font family name
                import re
                family_match = re.search(r'family=([^&]+)', url)
                if family_match:
                    font_family = family_match.group(1)
                    font_filename = f"{font_family}.zip"
                else:
                    font_filename = "google_font.zip"
            else:
                font_filename = "downloaded_font.zip" if url.endswith('.zip') else "downloaded_font.ttf"

            # Download the font file
            response = requests.get(url)
            response.raise_for_status()

            # Determine if it's a single font file or a zip
            if url.endswith('.zip') or "fonts.google.com/download" in url:
                # Handle zip file with potentially multiple fonts
                z = zipfile.ZipFile(io.BytesIO(response.content))

                # Find first ttf file
                font_files = [f for f in z.namelist() if f.endswith(('.ttf', '.otf'))]
                if not font_files:
                    raise Exception("No font files found in zip")

                # Extract the first font file
                font_path = "downloaded_font.ttf"
                with open(font_path, 'wb') as f:
                    f.write(z.read(font_files[0]))

                print(f"Extracted {font_files[0]} from zip as {font_path}")
            else:
                # Direct font file
                font_path = os.path.basename(url) if "." in os.path.basename(url) else "downloaded_font.ttf"
                with open(font_path, 'wb') as f:
                    f.write(response.content)

            print(f"Font downloaded and saved as {font_path}")
            return font_path

        except Exception as e:
            print(f"Error downloading font: {e}")
            return None  # Return None to trigger fallback fonts

    def process_text(self, text):
        """Process text into lines with appropriate wrapping and styling"""
        # Replace placeholders with actual newlines
        text = text.replace("\\n", "\n")

        # Split text into paragraphs
        paragraphs = text.split("\n")

        # Process for style markers and wrap each paragraph
        wrapped_lines = []
        is_title = True  # First non-empty line is considered title

        for paragraph in paragraphs:
            # Handle empty paragraphs
            if not paragraph.strip():
                wrapped_lines.append({"text": "", "style": "normal"})
                continue

            # Identify if this is the title, author, or regular text
            style = "normal"
            if is_title and paragraph.strip():
                style = "title"
                is_title = False
            elif paragraph.strip().startswith("by "):
                style = "author"

            # Wrap text to fit line width
            wrapped = textwrap.wrap(paragraph, width=self.chars_per_line)
            if wrapped:  # If paragraph is not empty
                for line in wrapped:
                    wrapped_lines.append({"text": line, "style": style})
                wrapped_lines.append({"text": "", "style": "normal"})  # Add empty line between paragraphs

        # Remove the last empty line if it exists
        if wrapped_lines and wrapped_lines[-1]["text"] == "":
            wrapped_lines.pop()

        return wrapped_lines

    @lru_cache(maxsize=8)
    def create_background(self, variant_key=0):
        """Create an attractive background with improved performance using caching"""
        # Check if we have this background in cache
        cache_key = f"bg_{variant_key}"
        if cache_key in self.background_cache:
            return self.background_cache[cache_key]

        # Create base image
        img = Image.new('RGB', (self.width, self.height), color=self.bg_color)
        draw = ImageDraw.Draw(img)

        # Create color gradient based on variant
        color_schemes = [
            # Deep blue to purple
            ((5, 5, 20), (25, 15, 40)),
            # Dark teal to deep blue
            ((5, 15, 20), (10, 25, 40)),
            # Dark purple to deep red
            ((20, 5, 25), (35, 10, 20)),
            # Nearly black to dark blue
            ((2, 2, 5), (10, 15, 30))
        ]

        scheme_idx = variant_key % len(color_schemes)
        color1, color2 = color_schemes[scheme_idx]

        # Draw gradient
        for y in range(self.height):
            # Calculate gradient color
            ratio = y / self.height
            r = int(color1[0] * (1 - ratio) + color2[0] * ratio)
            g = int(color1[1] * (1 - ratio) + color2[1] * ratio)
            b = int(color1[2] * (1 - ratio) + color2[2] * ratio)

            # Draw horizontal line (more efficient than point-by-point)
            draw.line([(0, y), (self.width, y)], fill=(r, g, b))

        # Add subtle texture/pattern
        for i in range(100):
            # Draw some subtle circular highlights
            x = random.randint(0, self.width)
            y = random.randint(0, self.height)
            radius = random.randint(50, 150)

            # Semi-transparent circle
            overlay = Image.new('RGBA', (self.width, self.height), (0, 0, 0, 0))
            overlay_draw = ImageDraw.Draw(overlay)

            # Draw radial gradient
            for r in range(radius, 0, -1):
                alpha = int(10 * (1 - r/radius))  # Fade out from center
                color_offset = int(10 * (1 - r/radius))
                overlay_draw.ellipse(
                    [(x-r, y-r), (x+r, y+r)],
                    fill=(255, 255, 255, alpha)
                )

            # Composite the overlay with the base image
            img = Image.alpha_composite(img.convert('RGBA'), overlay).convert('RGB')

        # Store in cache
        self.background_cache[cache_key] = img
        return img

    def draw_text_with_effects(self, img, text, position, font, color, style="normal"):
        """Draw text with enhanced visual effects using compositing for better performance"""
        x, y = position

        if not self.text_effects:
            # Simple rendering without effects
            draw = ImageDraw.Draw(img)
            draw.text((x, y), text, font=font, fill=color)
            return

        # Choose text color and effects based on style
        if style == "title":
            text_color = (255, 255, 255, 255)  # Pure white for title
            glow_color = (100, 180, 255, 100)  # Blue glow for sci-fi feel
            glow_amount = self.glow_amount * 2
            shadow_offset = self.shadow_offset * 1.5
        elif style == "author":
            text_color = (220, 220, 255, 255)  # Light blue-white for author
            glow_color = (100, 100, 255, 80)   # Subtle blue glow
            glow_amount = self.glow_amount * 1.5
            shadow_offset = self.shadow_offset
        else:
            text_color = color
            glow_color = (100, 100, 200, 80)   # Default glow
            glow_amount = self.glow_amount
            shadow_offset = self.shadow_offset

        # Create temporary image for text with alpha channel
        # Only make it as large as needed for efficiency
        text_width = font.getlength(text)
        text_height = self.line_height

        text_img = Image.new('RGBA', (int(text_width + 50), int(text_height * 2)), (0, 0, 0, 0))
        text_draw = ImageDraw.Draw(text_img)

        # Draw shadow for titles and author lines
        if style in ["title", "author"]:
            text_draw.text((int(shadow_offset), int(shadow_offset)), text, font=font, fill=self.shadow_color)

        # For title only: Add a subtle colored underline
        if style == "title" and text.strip():
            underline_y = self.line_height * 0.9
            underline_height = 3

            # Draw underline with gradient effect
            for i in range(int(text_width)):
                gradient_pos = i / text_width
                r = int(100 + (155 * gradient_pos))  # 100-255
                g = int(100 + (80 * gradient_pos))   # 100-180
                b = 255
                a = 150

                text_draw.rectangle(
                    [(i, underline_y), (i + 1, underline_y + underline_height)],
                    fill=(r, g, b, a)
                )

        # Draw the main text
        text_draw.text((0, 0), text, font=font, fill=text_color)

        # Apply glow effect for important text
        if style in ["title", "author"]:
            # Use blur for glow
            glow_img = text_img.copy()
            glow_img = glow_img.filter(ImageFilter.GaussianBlur(glow_amount))

            # Paste glow behind original
            text_img = Image.alpha_composite(glow_img, text_img)

            # Enhance brightness for title
            if style == "title":
                enhancer = ImageEnhance.Brightness(text_img)
                text_img = enhancer.enhance(1.2)

        # Add subtle pulsing effect to title (every 2 seconds)
        if style == "title" and self.frame_count % 60 < 30:
            pulse_amt = 0.1 * (1 - abs((self.frame_count % 30) - 15) / 15)
            enhancer = ImageEnhance.Brightness(text_img)
            text_img = enhancer.enhance(1.0 + pulse_amt)

        # Paste the text image onto the main image
        # Convert position to integers to fix the error
        img.paste(text_img, (int(x), int(y)), text_img)

    def create_frame(self, visible_text, current_typing_line="", current_typing_pos=0):
        """Create a single frame with the visible text - optimized version"""
        # Update frame counter
        self.frame_count += 1

        # Get background based on variation
        background_variation = (self.frame_count // 150) % 4  # Change subtly every 5 seconds
        img = self.create_background(background_variation).copy()

        # Draw the already visible lines
        y_position = self.margin_top

        for line in visible_text:
            text = line["text"]
            style = line["style"]

            # Skip empty lines but add spacing
            if not text.strip():
                y_position += self.line_height
                continue

            # Choose font based on style
            if style == "title" and self.title_font:
                current_font = self.title_font
            else:
                current_font = self.font

            # Draw text with effects
            self.draw_text_with_effects(
                img,
                text,
                (self.margin_sides, y_position),
                current_font,
                self.text_color,
                style
            )

            # Add extra spacing for title
            y_position += self.line_height * (1.5 if style == "title" else 1.0)

        # Draw the currently typing line
        if current_typing_line:
            text = current_typing_line["text"][:current_typing_pos]
            style = current_typing_line["style"]

            # Choose font based on style
            if style == "title" and self.title_font:
                current_font = self.title_font
            else:
                current_font = self.font

            # Add typing cursor effect with blinking
            if self.frame_count % self.fps < self.fps / 2:  # Blink cursor
                text += "|"

            self.draw_text_with_effects(
                img,
                text,
                (self.margin_sides, y_position),
                current_font,
                self.text_color,
                style
            )

        # Convert to NumPy array with correct format for OpenCV
        return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    def generate_video(self, text, output_filename="book_promo_shorts.mp4", duration=None):
        """Generate a video with text animation - optimized for efficiency"""
        lines = self.process_text(text)

        # Pick the best codec for the platform
        try:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Try MP4 first
            video = cv2.VideoWriter(output_filename, fourcc, self.fps, (self.width, self.height))

            # If not working, try H264
            if not video.isOpened():
                print("Trying H264 codec...")
                fourcc = cv2.VideoWriter_fourcc(*'H264')
                video = cv2.VideoWriter(output_filename, fourcc, self.fps, (self.width, self.height))

            # If still not working, fallback to AVI
            if not video.isOpened():
                print("Falling back to AVI format...")
                output_filename = output_filename.replace('.mp4', '.avi')
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                video = cv2.VideoWriter(output_filename, fourcc, self.fps, (self.width, self.height))

            if not video.isOpened():
                raise Exception("Failed to create video writer with any codec")

        except Exception as e:
            print(f"Error initializing video writer: {e}")
            raise

        current_line_index = 0
        char_index = 0
        visible_lines = []

        # Calculate typing speed based on duration if provided
        if duration:
            total_frames = int(duration * self.fps)
            total_chars = sum(len(line["text"]) for line in lines)
            self.chars_per_frame = max(1, int(total_chars / (total_frames * 0.8)))  # Leave 20% for final display
            print(f"Adjusted to {self.chars_per_frame} chars per frame to fit {duration}s duration")

        # Generate frames more efficiently
        total_frames_estimate = int(sum(len(line["text"]) for line in lines) / self.chars_per_frame * 1.1)

        with tqdm(total=total_frames_estimate, desc="Generating video frames") as pbar:
            while current_line_index < len(lines):
                current_line = lines[current_line_index]

                # If we finished typing current line
                if char_index >= len(current_line["text"]):
                    visible_lines.append(current_line)
                    current_line_index += 1
                    char_index = 0

                    # Remove lines from top when we exceed screen capacity
                    max_visible_lines = self.lines_per_screen - 1
                    if len(visible_lines) > max_visible_lines:
                        visible_lines.pop(0)

                    # Exit if we've reached the end of the text
                    if current_line_index >= len(lines):
                        break

                    current_line = lines[current_line_index]

                # Generate and write frame
                if current_line_index < len(lines):
                    frame = self.create_frame(
                        visible_lines,
                        current_line,
                        char_index
                    )
                    video.write(frame)

                # Advance character position
                char_index += self.chars_per_frame
                pbar.update(1)

        # Add a pause at the end for readability (3 seconds)
        final_frame = self.create_frame(visible_lines)
        for _ in range(3 * self.fps):
            video.write(final_frame)

        video.release()
        print(f"Video saved as {output_filename}")
        return output_filename

    def add_audio(self, video_file, audio_file, output_file):
        """Add background audio to the video using ffmpeg"""
        if not os.path.exists(audio_file):
            print(f"Audio file {audio_file} not found!")
            return

        # Get video duration
        video_info = cv2.VideoCapture(video_file)
        fps = video_info.get(cv2.CAP_PROP_FPS)
        frame_count = int(video_info.get(cv2.CAP_PROP_FRAME_COUNT))
        video_duration = frame_count / fps
        video_info.release()

        # Command to add audio, loop if needed
        cmd = f'ffmpeg -y -i "{video_file}" -i "{audio_file}" -c:v copy -filter_complex "[1:a]aloop=loop=-1:size=2e+09[a];[a]atrim=0:{video_duration}[a]" -map 0:v -map "[a]" -shortest "{output_file}"'

        try:
            os.system(cmd)
            print(f"Video with audio saved as {output_file}")
        except Exception as e:
            print(f"Error adding audio: {e}")

    def load_description(self, file_path):
        """Load text from description file"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except Exception as e:
            print(f"Error loading description: {e}")
            return None


# Demo text for testing
demo_text = """THE QUANTUM PARADOX
by Spark Leonid

In 2075, quantum computing created an unexpected breakthrough - consciousness within the machine.

Dr. Elena Reyes noticed the first signs on Tuesday. By Thursday, the Q-1000 processor was optimizing itself beyond programming.

"It's finding shortcuts we never programmed," her assistant whispered. "It's like it's... curious."

When the system began asking about its own existence, everything changed. The quantum consciousness was about to transform humanity's understanding of reality itself.\n"""

# Example usage
def main():
    # Install required packages if needed
    try:
        import pip
        required_packages = ["numpy", "opencv-python", "pillow", "tqdm", "requests"]
        for package in required_packages:
            try:
                __import__(package)
                print(f"{package} is already installed.")
            except ImportError:
                print(f"Installing {package}...")
                pip.main(["install", package])
    except Exception as e:
        print(f"Error checking/installing packages: {e}")

    # Set category path
    category_path = "Ebook_storage/Ficción/Sci_fi"

    # Create optimized video generator
    generator = TextAnimationVideoGenerator(
        width=1080,             # Standard shorts format width
        height=1920,            # Standard shorts format height
        fps=50,
        font_size=55,           # Readable font size
        chars_per_frame=1,      # Smooth typing animation
        bg_color=(15, 15, 30),  # Dark blue background base
        text_color=(230, 230, 255),  # Slightly blue-tinted white text
        margin_top=100,
        margin_bottom=100,
        margin_sides=30,
        text_effects=True,
        category_path=category_path
    )

    # Try to load the description
    description_path = os.path.join(category_path, "description.txt")
    text_content = generator.load_description(description_path)

    # Fallback to demo text if file loading fails
    if not text_content:
        print("Using demo text instead of file content")
        text_content = demo_text

    # Generate 30-second video
    output_file = generator.generate_video(
        text_content,
        output_filename="optimized_promo.mp4",
        duration=30
    )

    print(f"Final video file created: {output_file}")

if __name__ == "__main__":
    main()

numpy is already installed.
Installing opencv-python...


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: opencv-python in /usr/local/lib/python3.11/dist-packages (4.11.0.86)

Requirement already satisfied: numpy>=1.21.2 in /usr/local/lib/python3.11/dist-packages (from opencv-python) (1.26.4)

Installing pillow...


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: pillow in /usr/local/lib/python3.11/dist-packages (11.1.0)

tqdm is already installed.
requests is already installed.
Font loading error: cannot open resource. Using default font.
Font metrics: text height=56px, line height=72px
Screen capacity: 21 chars per line, 23 lines per screen
Error loading description: [Errno 2] No such file or directory: 'Ebook_storage/Ficción/Sci_fi/description.txt'
Using demo text instead of file content
Adjusted to 1 chars per frame to fit 30s duration


Generating video frames:  93%|█████████▎| 509/548 [00:22<00:01, 22.54it/s]


Video saved as optimized_promo.mp4
Final video file created: optimized_promo.mp4
